# Simple network to minimize CRPS


The EMOS analog is a simple network like this:

![title](EMOS_network.png)

In this notebook we will build this simple network in theano and use the CRPS as a cost function. 

In [110]:
# First, let's import the libraries we need
import theano
import theano.tensor as T
import numpy as np
from importlib import reload   # So that we can reload utils
import utils; reload(utils)   # This contains our own functions
from utils import *

# Let's make this notebook reproducible by defining the random seed
np.random.RandomState(42)   # I don't even like the hitchhiker...

I followed this tutorial to figure out the theano basics: http://www.marekrei.com/blog/theano-tutorial/

We will now attempt to build a simplle class for our model following: https://github.com/marekrei/theano-tutorial/blob/master/classifier.py

So the first step is to create a class and initialize the network architecture. theano allocates a graph. This means that we first plot out the computations which will be done in the future. 

In [2]:
class EMOS_Network(object):
    def __init__(self):
        """
        This function is called once an object of this class is created.
        """
        # Before we start with the network, let's define
        # the learning rate as an input so we can vary it
        lr = T.fscalar('lr')
        
        # First let's define the input to the network
        # This is the ensemble mean (meanx), 
        # the ensemble stadnard deviation (stdx) and
        # the corresponding observation (target)
        # In theano we use tensors to describe these variables.
        # T.fvector allocates a float32 1D vector
        meanx = T.fvector('meanx')   # The name helps with debugging
        stdx = T.fvector('stdx')
        target = T.fvector('target')
        
        # Next we allocate the weights (a, b, c, d) as shared
        # variables and initialize some value for them.
        # For now we will just draw a random variable from N(0, 1)
        a = theano.shared(np.random.randn(), 'a')
        b = theano.shared(np.random.randn(), 'b')
        c = theano.shared(np.random.randn(), 'c')
        d = theano.shared(np.random.randn(), 'd')
        
        # Now that we have the input and the weights, 
        # we can set up the network.
        mu = a + meanx * b
        sigma = c + stdx * d
        
        # Now comes the cost function.
        # To stop sigma from becoming negative we first have to 
        # convert it the the variance and then take the square
        # root again. (I learned from experience...)
        # This part of the code is inspired by Kai Polsterer's code!
        var = T.sqr(sigma)
        # The following three variables are just for convenience
        loc = (target - mu) / T.sqrt(var)
        phi = 1.0 / np.sqrt(2.0 * np.pi) * T.exp(-T.square(loc) / 2.0)
        Phi = 0.5 * (1.0 + T.erf(loc / np.sqrt(2.0)))
        # First we will compute the crps for each input/target pair
        crps =  T.sqrt(var) * (loc * (2. * Phi - 1.) + 2 * phi - 1. / np.sqrt(np.pi))
        # Then we take the mean. The cost is now a scalar
        mean_crps = T.mean(crps)
        
        # Now compute the gradients of the cost function 
        # with respect to the four weights/parameters
        params = [a, b, c, d]   # Let's put them in a list for convenience
        gradients = theano.tensor.grad(mean_crps, params)
        
        # For gradient descent we now need to subtract the gradients
        # from our parameters to minimize the cost function
        # In theano we want to define a list of tuples containing
        # the old parameter and the updated parameter.
        updates = [(p, p - lr * g) for p, g in zip(params, gradients)]
        
        # So far no actual computations have been done. Now we will
        # define a Theano function, which takes input, does some 
        # calculations and returns some output. In our case, we use 
        # meanx, stdx and the target as an input plus the required 
        # learning rate and return the mean_crps
        # as an output. Then we tell the function to apply the update
        # every time it is called. This is the training
        self.train = theano.function([meanx, stdx, target, lr], 
                                     mean_crps, updates=updates)
        # Furthermore, we define a method for simply making a prediction
        # and returning the predicted values of mu and sigma
        # along with the mean_crps without updating the parameters
        self.predict = theano.function([meanx, stdx, target],
                                       [mu, sigma, mean_crps])

Now let's make a sample prediction for one day. First we need to prepare the data.

In [4]:
# DATA_DIR = '/project/meteo/w2w/C7/ppnn_data/'   # At LMU
DATA_DIR = '/Users/stephanrasp/repositories/ppnn/data/'  # Mac
fn = 'data_interpolated.nc'

In [105]:
# Let's load the full dataset for 00 UTC
tobs_full, tfc_full, dates = load_nc_data(DATA_DIR + fn, utc=0)

In [106]:
# Now let's load the data for a particular date
from datetime import datetime
# Let's pick a random date
date_idx = np.where(dates == datetime(2011, 2, 14, 0, 0))[0][0]

In [107]:
date_idx

1503

In [111]:
# Now let's get the data with the handy function in utils
window_size = 50
fclt = 48
tfc_mean_train, tfc_std_train, tobs_train, \
    tfc_mean_test, tfc_std_test, tobs_test = \
        get_train_test_data(tobs_full, tfc_full, date_idx, 
                            window_size, fclt)

In [112]:
# Now let's initialize the model
model = EMOS_Network()

In [113]:
# And train it
lr = np.asarray(0.1, dtype='float32')
for i in range(500):
    cost = model.train(tfc_mean_train, tfc_std_train, tobs_train, lr)
    if i%20 == 0: print('Step %i; mean_crps = %.3f' % (i, cost))

Step 0; mean_crps = 3.167
Step 20; mean_crps = 2.227
Step 40; mean_crps = 1.668
Step 60; mean_crps = 1.393
Step 80; mean_crps = 1.272
Step 100; mean_crps = 1.222
Step 120; mean_crps = 1.201
Step 140; mean_crps = 1.191
Step 160; mean_crps = 1.187
Step 180; mean_crps = 1.185
Step 200; mean_crps = 1.184
Step 220; mean_crps = 1.183
Step 240; mean_crps = 1.183
Step 260; mean_crps = 1.183
Step 280; mean_crps = 1.183
Step 300; mean_crps = 1.183
Step 320; mean_crps = 1.183
Step 340; mean_crps = 1.182
Step 360; mean_crps = 1.182
Step 380; mean_crps = 1.182
Step 400; mean_crps = 1.182
Step 420; mean_crps = 1.182
Step 440; mean_crps = 1.182
Step 460; mean_crps = 1.182
Step 480; mean_crps = 1.182


Ok, that seems to work. For this case the CRPS stops decreasing after about 200 steps. Now let's make a prediction for the date we picked.

In [114]:
model.predict(tfc_mean_test, tfc_std_test, tobs_test)[2]

array(0.77613490350798)

In [117]:
# Let's compare that to the skill of the raw ensemble
# Now rememeber tfc_mean_test and tfc_std_test are scaled,
# so they are not compatible to tobs
# So let's load the not-scaled (ns) data
tobs_test, tfc_mean_test_ns, tfc_std_test_ns = prep_data(tobs_full[date_idx],
                                                         tfc_full[date_idx])
crps_normal(tfc_mean_test_ns, tfc_std_test_ns, tobs_test).mean()

1.0212902697471031

So a definite improvement! Let's now set up a loop to go through the period. Then let's check what takes time and maybe try running this on GPU!